In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

import sqlite3

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.neighbors import KDTree

### Reading in the data

In [2]:
conn = sqlite3.connect("market_maven.sqlite")
raw_data = pd.read_sql("select * from metrics_no_percentiles_v", conn)
conn.close()

### Data cleaning and imputing missing values

In [3]:
# dropping columns that are just names or identifiers
numeric_data = raw_data.select_dtypes(exclude="object")

# imputing missing values based on nearest neighbors instead of deleting them
imp = KNNImputer(n_neighbors=10, weights="distance")
numeric_imputed = pd.DataFrame(imp.fit_transform(numeric_data), columns=numeric_data.columns)

### Removing outliers

In [4]:
# removing extreme outliers
# documentation says RobustScaler handles outliers but there was data for Rio Arriba county New Mexico and
# Loving county Texas that were too extreme so using StandardScaler and handling outliers manually
outliers = numeric_imputed[(np.abs(stats.zscore(numeric_imputed)) >= 50).all(axis=1)].index
if len(outliers) != 0:
    numeric_ex_outlier = numeric_imputed.drop(outliers).reset_index()
    full_ex_outlier = raw_data.drop(outliers).reset_index()
else:
    numeric_ex_outlier = numeric_imputed
    full_ex_outlier = raw_data

### Scaling data

In [5]:
scaler = StandardScaler().fit(numeric_ex_outlier) 
scaled_data = scaler.transform(numeric_ex_outlier)

### Fitting pca

In [6]:
my_pca = PCA().fit(scaled_data)

### Explained variance ratio of principal components

The first 18, of the 32 total principal components, explain roughly 99% of the variance in the data so they will be the focus

In [7]:
np.sum(my_pca.explained_variance_ratio_[:2])

0.417978974656947

In [115]:
components_df = pd.DataFrame(my_pca.components_, columns=numeric_data.columns)
top_components_df = components_df.iloc[:18,:].transpose()

### Coefficients analysis

The coefficients for each of the 32 attributes across the first 18 principal components are weighted by that components explained variance ratio and summed up

In [116]:
coef_sum = {}
i = 0
for col in top_components_df:
    if i == 0:
        # taking the absolute value, the magnitude can be large irrespective of the direction
        for ind, val in zip(top_components_df[col].abs().index, top_components_df[col].abs().values):
            coef_sum[ind] = val * my_pca.explained_variance_ratio_[i]
    else:
        # taking the absolute value, the magnitude can be large irrespective of the direction
        for ind, val in zip(top_components_df[col].abs().index, top_components_df[col].abs().values):
            coef_sum[ind] += val * my_pca.explained_variance_ratio_[i]
    i += 1

### Results

The results are stored in a DataFrame and sorted from largest to smallest

In [117]:
coef_sum_df = pd.DataFrame([coef_sum]).transpose()
coef_sum_df.columns = ["coef_sum"]
coef_sum_df.sort_values(by="coef_sum", ascending=False, inplace=True)
coef_sum_df

,coef_sum
edu_bachelors_and_higher_perc,0.146829
commute_green_perc,0.144784
popden_urban,0.143723
housing_vacancy_perc,0.137040
housing_occupied_perc,0.137039
popden_rural,0.136403
political_scale,0.135702
median_age,0.131015
total_hh,0.130736
tot_pop,0.130439


### Conclusion

In theory, these results can be used in conjunction with the results of our participant survey experiment. In that experiment, participants largely agreed that the results of our clustering algorithm and the similar counties it recommended were interesting to them. Those recommendations are driven by the principal components described above, particularly the first two as our visualization was two dimensional. Therefore, the results of summing up the weighted coefficients could be used to identify valuable data in a methodical and efficient manner and to guide the selection of features that would provide the most value to users of Market Maven.